# <center>How **Fuzzy Text Search** Works<br><small>Real-time search in 40 lines of Python</small></center>

<br>

## **Introduction**
- Fuzzy autocomplete for 60,000 cities in the UK <span style="color:#888">(using <code style="color:#888">rapidfuzz</code>, 2 lines of code)</span>
- `fuzzywuzzy`, `rapidfuzz` and `difflib` libraries in Python

## **How does it work?**

- When are two strings "similar"?
- Hamming distance
- Levenshtein distance, dynamic programming approach (Wagner-Fischer)
- Vectorization, closing the gap between Python and C
- Fuzzy autocomplete for 60,000 cities in the UK <span style="color:#888">(from scratch, 40 lines of code)</span>


<br class="vtab">

---

In [1]:
from fuzzywuzzy import fuzz as fuzzywuzzy_fuzz, process as fuzzywuzzy_process
from rapidfuzz import fuzz as rapidfuzz_fuzz, process as rapidfuzz_process
import Levenshtein

import pandas as pd
import numpy as np
from timeit import default_timer as timer
from functools import wraps
import ipywidgets as widgets

def print_timing(f):
    @wraps(f)
    def inner(*args, **kwargs):
        t0 = timer()
        try:
            return f(*args, **kwargs)
        finally:
            print(f"\n{f.__name__} finished in {1e3*(timer()-t0):.1f} ms")
    return inner

---

# <center>Introduction</center>

<br>

Let's say you want autocomplete for cities in Great Britain:

In [2]:
#    You can download this dataset here:
# 📥 https://geoportal.statistics.gov.uk/datasets/a6c138d17ac54532b0ca8ee693922f10_0

df = pd.read_csv("Index_of_Place_Names_in_Great_Britain_(July_2016).csv")
cities = df["place15nm"].unique()

@print_timing
def interact_cities(input_text):
    if not input_text: return
    for city, score, _ in rapidfuzz_process.extract(input_text, cities, limit=10):
        print(f"{city:60}{score:3.2f}")

widgets.interact(interact_cities, input_text="");

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

<br class="vtab">

**"Fuzzy" (approximate)** string matching looks for strings that are **similar** to a given string. Here are some great libraries you can use for this task in Python:

- **`fuzzywuzzy`**, a fuzzy text matching library ported to many languages, including JS ([GitHub](https://github.com/seatgeek/fuzzywuzzy))
- **`rapidfuzz`**, a performance-oriented version of `fuzzywuzzy` ([GitHub](https://github.com/maxbachmann/rapidfuzz))
- **`difflib`** from Python standard library ([docs](https://docs.python.org/3/library/difflib.html))

<br><br><br>

All of these provide functions to compute "similarity" of strings:

In [3]:
fuzzywuzzy_fuzz.ratio("Tokyo and Osaka", "Tokio and Osaka")

93

And, based on that, to find best matching strings for given input:

In [4]:
words = ["knight", "knuth", "nigh", "ignite", "knighthood", "knead", "the"]
fuzzywuzzy_process.extract("knigth", words)

[('knight', 83), ('nigh', 80), ('knighthood', 75), ('knuth', 73), ('the', 72)]

<br><br><br>

You can choose from different **ranking** functions or even create your own. The British cities example uses `fuzz.WRatio`, a mix of different ranking approaches (see [`fuzz.py:224`](https://github.com/seatgeek/fuzzywuzzy/blob/9e3d2fe0d8c1b195696d5fbcda78c371dd4a6b8f/fuzzywuzzy/fuzz.py#L224)).

- The search is made up from common **building blocks** to judge similarity.
- In this tutorial, we'll implement some of them **from scratch using Python**
- and **optimize them** so that we get something comparable to the first example:
- fuzzy search in 60K strings in "real time" (<100ms).

Note that `rapidfuzz` uses optimized C++ routines, so approaching its runtime performance will require some tricks.

---

<br class="vtab">

# <center>How does it work?</center>
<br><br>

### String similarity

When two strings are kind of similar, we need to go beyond `True`/`False`.

In [5]:
"mancesther" == "manchester"

False

In [6]:
fuzzywuzzy_fuzz.ratio("mancesther", "manchester")

90

How do you measure **similarity** of two strings **a**, **b**?
- Number of editing operations to get from **a** to **b**
    - Replace (Hamming)
    - Replace, insert, delete (Levenshtein)
    - Replace, insert, delete, transpose (Levenshtein-Damerau)
    - ...
- Longest common substring
- Common word count
- Letter frequency
- ...

<br>

Depending on the application (are we comparing single words? paragraphs? DNA sequences?), some similarity measures will be more appropriate than others. In this tutorial, we'll focus on **edit distance** measures.

<br class="vtab">

### Hamming distance

**Hamming distance** is the number of letters one has to **replace** to get from string **a** to string **b**.

- `d("cat", "hat") = 1`, replace 1st letter (`c -> h`)
- `d("cat", "lag") = 2`, replace 1st letter (`c -> l`) and 3rd letter (`t -> g`)
- `d("cat", "cats")` is conventionally not defined (it cannot be achieved by replacing letters)

<br>

Hamming distance is used in study of error-correcting codes (encoding schemes that are resilient to some number of corrupt bits). Its major **limitation** in that it doesn't handle *misaligned* sequences well.

<pre>
d("Hamming <b>distance</b>",
  "Hamming<b>distance</b> ") = 9
          <span style="color:red">xxxxxxxxx</span>
</pre>

<br class="vtab">

Let's try implementing it in Python:

In [7]:
def hamming_distance(a: str, b: str):
    if len(a) == len(b):
        return sum(1 for a_i, b_i in zip(a, b) if a_i != b_i)
    else:
        return float("inf")  # cannot get different length


@print_timing
def interact_cities_hamming(input_text):
    if not input_text: return
    for score, city in list(sorted((hamming_distance(input_text, s), s) for s in cities))[:10]:
        print(f"{city:60}{score:3.2f}")

widgets.interact(interact_cities_hamming, input_text="");

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

<br class="vtab">

### Levenshtein distance

**Hamming distance** is useful in specific contexts (like error-correcting codes), but for natural language, it's not very suitable. We'd like a more robust edit distance.

**Levenshtein distance** is widely used for this reason. It has these operations:
- Substitution (like Hamming)
- Insertion
- Deletion

<br>The distance is defined as the *minimal* number of edits. There may be multiple minimal edits.

- `d("cat", "cats") = 1`, as in `cats -> cat`
- `d("moon", "monsoon") = 3`, as in `monsoon -> mnsoon -> msoon -> moon`
- `d("knight", "knigth") = 2`, as in `knigth -> knigh -> knight`

Reading both strings from the left, *it's not obvious* which operation should be chosen at any given position. It's tempting to do nothing when characters match, but deleting or inserting characters instead may better align the remaining portion.

<pre>
d("Hamming <b>distance</b>",
  "Hamming<b>distance</b>") = 1
          <span style="color:red">insert " ", not replace "d" -> " "!</span>
</pre>

<br>

How to solve this conundrum?

- We can simply try all possibilities using **recursion**...
- ...which can be optimized using **dynamic programming** into the [Wagner-Fischer algorithm](https://en.wikipedia.org/wiki/Wagner%E2%80%93Fischer_algorithm).
- An alternative approach is to use **finite automata**.


<br class="vtab">

### Levenshtein distance with DP (Wagner-Fischer)

For brevity, we'll only discuss one algorithm for Levenshtein distance &mdash; the [Wagner-Fischer algorithm](https://en.wikipedia.org/wiki/Wagner%E2%80%93Fischer_algorithm), which runs in $\mathcal{O}(n^2)$ time and memory space for words of length $n$.

In [8]:
def levenshtein_distance(a: str, b: str, verbose: bool = False) -> int:
    m, n = len(a), len(b)
    d = np.zeros((m+1, n+1), dtype=int)  # d[i,j] = levenshtein_distance(a[:i], b[:j])
        
    # when the other string is empty, distance is length of non-empty string
    for i in range(m+1): d[i, 0] = i
    for i in range(n+1): d[0, i] = i
    
    for j in range(1, n+1):
        for i in range(1, m+1):
            cost = 1 if a[i-1] != b[j-1] else 0
            d[i, j] = min(d[i-1, j-1] + cost,   # substitute         ↘
                          d[i, j-1]   + 1,      # delete from B      →
                          d[i-1, j]   + 1)      # insert into B      ↓
    
    if verbose: print(d)
    return d[m, n]

In [9]:
d = levenshtein_distance("cat", "wildcat", verbose=True)
d

[[0 1 2 3 4 5 6 7]
 [1 1 2 3 4 4 5 6]
 [2 2 2 3 4 5 4 5]
 [3 3 3 3 4 5 5 4]]


4

<br class="vtab">

### Can we search in real time now?

Compared to optimized C implementation, our `levenshtein_distance()` is **100-500x slower**.

This is to be expected from this kind of code in Python.

In [10]:
# optimized C implementation, https://pypi.org/project/python-Levenshtein
%timeit Levenshtein.distance("cat", "rat")
%timeit Levenshtein.distance("knight", "knigth")
%timeit Levenshtein.distance("unimaginable", "imagination")

112 ns ± 7.32 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
142 ns ± 3.09 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
307 ns ± 3.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%timeit levenshtein_distance("cat", "rat")
%timeit levenshtein_distance("knight", "knigth")
%timeit levenshtein_distance("unimaginable", "imagination")

12.9 µs ± 54.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
44.6 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
167 µs ± 6.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<br>
  
Let's try the British cities example with this implementation:

In [12]:
@print_timing
def interact_cities_levenshtein(input_text):
    n = len(input_text)
    def rank(s):
        m = len(s)
        if abs(m - n) > 5:  # 🔧 cheating a bit - we'll skip words which would have
            return 0        #                     a large distance anyway
        elif m == n:
            dist = hamming_distance(input_text, s)  # 🔧 not cheating, gives the same results
        else:
            dist = levenshtein_distance(input_text, s)
        return 1.0 - dist/max(n, m)
        
    if not input_text: return
    for score, city in list(sorted(((rank(s), s) for s in cities), reverse=True))[:10]:
        print(f"{city:60}{100*score:3.2f}")

widgets.interact(interact_cities_levenshtein, input_text="");

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

<br class="vtab">

### Vectorization

Our Wagner-Fischer in Python is close, but not quite there for real-time querying. One possible thing to do here would be to optimize the function as it is, eg. using **Cython** or **Numba**.

However, there is a different way &mdash; we can **compute many distances at once**, using the exact same logic. Indeed, for all strings of the same length, the algorithm does exactly the same steps, just the values in the table are different.

Instead of one 2D table, we can have a "3D table", really just many 2D tables stacked together. Then `d[i,j]` will not be just one number, but a **vector** holding the results for all processed strings at once.

This should be significantly faster, because the operations we need (sum, comparison, minimum) can be executed very efficiently for vectors using eg. NumPy. **The Python function itself will be as slow as ever, but we'll only need to call it once**, not thousands of times. Note that this doesn't change the asymptotic running time at all, it's just about the efficiency of Python vs. native code.


In [13]:
def levenshtein_distance_vec(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    m, n, k = len(a), len(b), b.shape[1]
    d = np.zeros((m+1, n+1, k), dtype=np.uint16)  # d[i,j] = levenshtein_distance(a[:i], b[:j])

    # when the other string is empty, distance is length of non-empty string
    for i in range(m+1): d[i, 0] = i
    for i in range(n+1): d[0, i] = i
    
    for j in range(1, n+1):
        for i in range(1, m+1):
            cost = a[i-1] != b[j-1]
            d[i, j] = np.min([d[i-1, j-1] + cost,        # substitute         ↘
                              d[i, j-1]   + 1,           # delete from B      →
                              d[i-1, j]   + 1], axis=0)  # insert into B      ↓

    return d[m, n]

In [14]:
# convert input strings to NumPy matrices
len_to_cities = {}
for w in cities: len_to_cities.setdefault(len(w), []).append(w)
len_to_mat = {n: np.asarray([[ord(c) for c in w] for w in ws], dtype=np.uint16).T
              for n, ws in len_to_cities.items()}

@print_timing
def interact_cities_levenshtein_vec(input_text):
    a_vec = np.asarray([ord(c) for c in input_text])
    n = len(input_text)
    xs, ys = [], []
    
    for m, b_mat in len_to_mat.items():
        if abs(m - n) > 5: continue  # 🔧 again, skipping strings with obvious large distance
        dist = levenshtein_distance_vec(a_vec, b_mat)
        scores = 1.0 - dist/max(n, m)
        xs.extend(len_to_cities[m])
        ys.extend(scores) 
    
    if not input_text: return
    for score, city in list(sorted(zip(ys, xs), reverse=True))[:10]:
        print(f"{city:60}{score:3.2f}")

widgets.interact(interact_cities_levenshtein_vec, input_text="");

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

<br><br>
Finally, let's compare the perfomance with Levenshtein in C:

In [15]:
a = "londen"
a_vec = np.asarray([ord(c) for c in a])

print("levenshtein_distance")
%timeit -r 1 -n 1 [levenshtein_distance(a, b) for b in cities]
print("\nlevenshtein_distance_vec")
%timeit [levenshtein_distance_vec(a_vec, b_mat) for b_mat in len_to_mat.values()]
print("\nC implementation")
%timeit [Levenshtein.distance(a, b) for b in cities]

levenshtein_distance
5.47 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

levenshtein_distance_vec
153 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

C implementation
17.8 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


While the initial implementation is ~300x slower, the vectorized one is just ~10x slower.